Langmuir-Hinshelwood model: Acceleration by Automated Rescaling of the Rate Constants.
--------------------------------------------------------------------------------------



In [1]:
import multiprocessing
import numpy
import scm.plams
import scm.pyzacros as pz
import scm.pyzacros.models

In [2]:
scm.pyzacros.init()

PLAMS working folder: /home/aguirre/Develop/pyzacros/examples/ZiffGulariBarshad/plams_workdir.012


In [3]:
maxjobs = multiprocessing.cpu_count()
scm.plams.config.default_jobrunner = scm.plams.JobRunner(parallel=True, maxjobs=maxjobs)
scm.plams.config.job.runscript.nproc = 1
print('Running up to {} jobs in parallel simultaneously'.format(maxjobs))

Running up to 8 jobs in parallel simultaneously


In [4]:
lh = pz.models.LangmuirHinshelwood()

In [5]:
dt = 1.0e-5
z_sett = pz.Settings()
z_sett.random_seed = 1609
z_sett.temperature = 500.0
z_sett.pressure = 1.000
z_sett.species_numbers = ('time', dt)
z_sett.max_time = 100*dt

z_job = pz.ZacrosJob( settings=z_sett, lattice=lh.lattice,
                      mechanism=lh.mechanism,
                      cluster_expansion=lh.cluster_expansion )

In [6]:
ss_sett = pz.Settings()
ss_sett.turnover_frequency.nbatch = 20
#ss_sett.turnover_frequency.confidence = 0.98
ss_sett.turnover_frequency.confidence = 0.5
ss_sett.turnover_frequency.ignore_nbatch = 5
ss_sett.nreplicas = 4
ss_sett.scaling.enabled = 'T'
ss_sett.scaling.partial_equilibrium_index_threshold = 0.1
ss_sett.scaling.upper_bound = 100
ss_sett.scaling.max_time = 10*dt

ss_params = pz.ZacrosSteadyStateJob.Parameters()
ss_params.add( 'max_time', 'restart.max_time',
                2*z_sett.max_time*( numpy.arange(50)+1 )**2 )

ss_job = pz.ZacrosSteadyStateJob( settings=ss_sett, reference=z_job,
                                  parameters=ss_params )

[17.01|22:45:51] JOB plamsjob Steady State Convergence: Using nbatch=20,confidence=0.5,ignore_nbatch=5,nreplicas=4


In [7]:
ps_params = pz.ZacrosParametersScanJob.Parameters()
#ps_params.add( 'x_CO', 'molar_fraction.CO', numpy.linspace(0.0, 1.0, 21) )
ps_params.add( 'x_CO', 'molar_fraction.CO', numpy.linspace(0.0, 1.0, 3) )
ps_params.add( 'x_O2', 'molar_fraction.O2', lambda params: 1.0-params['x_CO'] )

ps_job = pz.ZacrosParametersScanJob( reference=ss_job, parameters=ps_params )

[17.01|22:45:51] JOB plamsjob_ps_cond000 Steady State Convergence: Using nbatch=20,confidence=0.5,ignore_nbatch=5,nreplicas=4
[17.01|22:45:51] JOB plamsjob_ps_cond001 Steady State Convergence: Using nbatch=20,confidence=0.5,ignore_nbatch=5,nreplicas=4
[17.01|22:45:51] JOB plamsjob_ps_cond002 Steady State Convergence: Using nbatch=20,confidence=0.5,ignore_nbatch=5,nreplicas=4


In [8]:
results = ps_job.run()

if not ps_job.ok():
    print('Something went wrong!')

[17.01|22:45:51] JOB plamsjob STARTED
[17.01|22:45:51] Waiting for job plamsjob to finish
[17.01|22:45:51] JOB plamsjob RUNNING
[17.01|22:45:51] JOB plamsjob/plamsjob_ps_cond000 STARTED
[17.01|22:45:51] JOB plamsjob/plamsjob_ps_cond001 STARTED
[17.01|22:45:51] JOB plamsjob/plamsjob_ps_cond002 STARTED
[17.01|22:45:51] JOB plamsjob/plamsjob_ps_cond000 RUNNING
[17.01|22:45:51] JOB plamsjob/plamsjob_ps_cond000 Scaling: Using partial_equilibrium_index_threshold=0.1,upper_bound=100,max_time=0.0001
[17.01|22:45:51] JOB plamsjob/plamsjob_ps_cond001 RUNNING
[17.01|22:45:51] JOB plamsjob/plamsjob_ps_cond000/plamsjob_ps_cond000_ss_scaling STARTED
[17.01|22:45:51] JOB plamsjob/plamsjob_ps_cond002 RUNNING
[17.01|22:45:51] JOB plamsjob/plamsjob_ps_cond001 Scaling: Using partial_equilibrium_index_threshold=0.1,upper_bound=100,max_time=0.0001
[17.01|22:45:51] Waiting for job plamsjob_ps_cond000_ss_scaling to finish
[17.01|22:45:51] JOB plamsjob/plamsjob_ps_cond002 Scaling: Using partial_equilibrium_in

In [9]:
x_CO = []
ac_O = []
ac_CO = []
TOF_CO2 = []

results_dict = results.turnover_frequency()
results_dict = results.average_coverage( last=10, update=results_dict )

for i in range(len(results_dict)):
    x_CO.append( results_dict[i]['x_CO'] )
    ac_O.append( results_dict[i]['average_coverage']['O*'] )
    ac_CO.append( results_dict[i]['average_coverage']['CO*'] )
    TOF_CO2.append( results_dict[i]['turnover_frequency']['CO2'] )

print( '------------------------------------------------' )
print( '%4s'%'cond', '%8s'%'x_CO', '%10s'%'ac_O', '%10s'%'ac_CO', '%12s'%'TOF_CO2' )
print( '------------------------------------------------' )
for i in range(len(x_CO)):
    print( '%4d'%i, '%8.2f'%x_CO[i], '%10.6f'%ac_O[i], '%10.6f'%ac_CO[i], '%12.6f'%TOF_CO2[i] )

------------------------------------------------
cond     x_CO       ac_O      ac_CO      TOF_CO2
------------------------------------------------
   0     0.00   0.678625   0.000000     0.000000
   1     0.50   0.444906   0.279719   330.208333
   2     1.00   0.000000   0.627156     0.000000


In [10]:
scm.pyzacros.finish()

[17.01|22:46:45] PLAMS run finished. Goodbye


$$
\theta_\text{CO} = \frac{ K_\text{CO}P_\text{CO} }{ 1 + K_\text{CO}P_\text{CO} + \sqrt{K_{\text{O}_2}P_\text{O}} } \\
\theta_\text{O} = \frac{ \sqrt{K_{\text{O}_2}P_\text{O}} }{ 1 + K_\text{CO}P_\text{CO} + \sqrt{K_{\text{O}_2}P_\text{O}} } \\[7mm]
\text{TOF}_{\text{CO}_2} = 6 \, k_\text{oxi}\theta_\text{CO}\theta_\text{O}
$$